In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import os.path
import time

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import cifar10, cifar_input, cifar_input_ver2

FLAGS = tf.app.flags.FLAGS

'''tf.app.flags.DEFINE_string('train_dir', '/tmp/cifar10VGG_train',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 25000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")'''

IMAGE_SIZE = 24

# Global constants describing the CIFAR-10 data set.
NUM_CLASSES = 10
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000


# Constants describing the training process.
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.1       # Initial learning rate.

max_steps = 40000
train_dir = 'cifar10VGG_train/'
batch_size = 128
log_device_placement = False

In [ ]:
def loss(logits, labels):
  """Add L2Loss to all the trainable variables.

  Add summary for "Loss" and "Loss/avg".
  Args:
    logits: Logits from inference().
    labels: Labels from distorted_inputs or inputs(). 1-D tensor
            of shape [batch_size]

  Returns:
    Loss tensor of type float.
  """
  # Calculate the average cross entropy loss across the batch.
  # labels = tf.cast(labels, tf.int64)
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
          logits=logits, labels=labels, name='cross_entropy_per_example')
  cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
  tf.add_to_collection('losses', cross_entropy_mean)

  # The total loss is defined as the cross entropy loss plus all of the weight
  # decay terms (L2 loss).
  return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [ ]:
def _add_loss_summaries(total_loss):
  """Add summaries for losses in CIFAR-10 model.

  Generates moving average for all losses and associated summaries for
  visualizing the performance of the network.

  Args:
    total_loss: Total loss from loss().
  Returns:
    loss_averages_op: op for generating moving averages of losses.
  """
  # Compute the moving average of all individual losses and the total loss.
  loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
  losses = tf.get_collection('losses')
  loss_averages_op = loss_averages.apply(losses + [total_loss])

  return loss_averages_op

def train(total_loss, global_step):
  """Train CIFAR-10 model.

  Create an optimizer and apply to all trainable variables. Add moving
  average for all trainable variables.

  Args:
    total_loss: Total loss from loss().
    global_step: Integer Variable counting the number of training steps
      processed.
  Returns:
    train_op: op for training.
  """
  # Variables that affect learning rate.
  num_batches_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / batch_size
  decay_steps = int(num_batches_per_epoch * NUM_EPOCHS_PER_DECAY)

  # Decay the learning rate exponentially based on the number of steps.
  lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
                                  global_step,
                                  decay_steps,
                                  LEARNING_RATE_DECAY_FACTOR,
                                  staircase=True)

  # Generate moving averages of all losses and associated summaries.
  loss_averages_op = _add_loss_summaries(total_loss)

  # Compute gradients.
  with tf.control_dependencies([loss_averages_op]):
    opt = tf.train.GradientDescentOptimizer(lr)
    grads = opt.compute_gradients(total_loss)

  # Apply gradients.
  apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

  # Track the moving averages of all trainable variables.
  variable_averages = tf.train.ExponentialMovingAverage(
      MOVING_AVERAGE_DECAY, global_step)
  variables_averages_op = variable_averages.apply(tf.trainable_variables())

  with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
    train_op = tf.no_op(name='train')

  return train_op

In [ ]:
def inference(images):
  """Build the CIFAR-10 model.

  Args:
    images: Images returned from distorted_inputs() or inputs().

  Returns:
    Logits.
  """
  # We instantiate all variables using tf.get_variable() instead of
  # tf.Variable() in order to share variables across multiple GPU training runs.
  # If we only ran this model on a single GPU, we could simplify this function
  # by replacing all instances of tf.get_variable() with tf.Variable().
  #
  # conv1

  # zero-mean input
  # conv1_1
  parameters = []
  with tf.name_scope('conv1_1') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv1_1 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # conv1_2
  with tf.name_scope('conv1_2') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv1_2 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # pool1
  pool1 = tf.nn.max_pool(conv1_2,
                         ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1],
                         padding='SAME',
                         name='pool1')

  # conv2_1
  with tf.name_scope('conv2_1') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv2_1 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # conv2_2
  with tf.name_scope('conv2_2') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv2_2 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # pool2
  pool2 = tf.nn.max_pool(conv2_2,
                         ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1],
                         padding='SAME',
                         name='pool2')

  # conv3_1
  with tf.name_scope('conv3_1') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv3_1 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # conv3_2
  with tf.name_scope('conv3_2') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv3_2 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # conv3_3
  with tf.name_scope('conv3_3') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv3_3 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # pool3
  pool3 = tf.nn.max_pool(conv3_3,
                         ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1],
                         padding='SAME',
                         name='pool3')

  # conv4_1
  with tf.name_scope('conv4_1') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(pool3, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv4_1 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # conv4_2
  with tf.name_scope('conv4_2') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv4_2 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # conv4_3
  with tf.name_scope('conv4_3') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv4_3 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # pool4
  pool4 = tf.nn.max_pool(conv4_3,
                         ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1],
                         padding='SAME',
                         name='pool4')

  # conv5_1
  with tf.name_scope('conv5_1') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(pool4, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv5_1 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # conv5_2
  with tf.name_scope('conv5_2') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv5_2 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # conv5_3
  with tf.name_scope('conv5_3') as scope:
      kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                               stddev=1e-1), name='weights')
      conv = tf.nn.conv2d(conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                           trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      conv5_3 = tf.nn.relu(out, name=scope)
      parameters += [kernel, biases]

  # pool5
  pool5 = tf.nn.max_pool(conv5_3,
                         ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1],
                         padding='SAME',
                         name='pool4')

  with tf.name_scope('fc1') as scope:
      shape = int(np.prod(pool5.get_shape()[1:]))
      fc1w = tf.Variable(tf.truncated_normal([shape, 4096],
                                                   dtype=tf.float32,
                                                   stddev=1e-1), name='weights')
      fc1b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
                           trainable=True, name='biases')
      pool5_flat = tf.reshape(pool5, [-1, shape])
      fc1l = tf.nn.bias_add(tf.matmul(pool5_flat, fc1w), fc1b)
      fc1 = tf.nn.relu(fc1l)
      parameters += [fc1w, fc1b]

  # fc2
  with tf.name_scope('fc2') as scope:
      fc2w = tf.Variable(tf.truncated_normal([4096, 4096],
                                                   dtype=tf.float32,
                                                   stddev=1e-1), name='weights')
      fc2b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
                           trainable=True, name='biases')
      fc2l = tf.nn.bias_add(tf.matmul(fc1, fc2w), fc2b)
      fc2 = tf.nn.relu(fc2l)
      parameters += [fc2w, fc2b]

  # fc3
  with tf.name_scope('fc3') as scope:
      fc3w = tf.Variable(tf.truncated_normal([4096, 10],
                                                   dtype=tf.float32,
                                                   stddev=1e-1), name='weights')
      fc3b = tf.Variable(tf.constant(1.0, shape=[10], dtype=tf.float32),
                           trainable=True, name='biases')
      fc3l = tf.nn.bias_add(tf.matmul(fc2, fc3w), fc3b)
      parameters += [fc3w, fc3b]

  return fc3l

In [ ]:
with tf.Graph().as_default():
    global_step = tf.contrib.framework.get_or_create_global_step()

    # Get images and labels for CIFAR-10.
    images, labels = cifar_input.build_input('cifar10', 'cifar10/data_batch*', batch_size, 'train')

    # Build a Graph that computes the logits predictions from the
    # inference model.
    logits = inference(images)

    # Calculate losses.
    losses = loss(logits, labels)

    # Build a Graph that trains the model with one batch of examples and
    # updates the model parameters.
    train_op = train(losses, global_step)

    class _LoggerHook(tf.train.SessionRunHook):
      """Logs losses and runtime."""

      def begin(self):
        self._step = -1
        f = open("log.txt",'ab')
        f.write('\n\n==== Run ===\nInfo: VGG\n')
        f.close()

      def before_run(self, run_context):
        self._step += 1
        self._start_time = time.time()
        return tf.train.SessionRunArgs(losses)  # Asks for losses value.

      def after_run(self, run_context, run_values):
        duration = time.time() - self._start_time
        losses_value = run_values.results
        if self._step % 10 == 0:
          num_examples_per_step = batch_size
          examples_per_sec = num_examples_per_step / duration
          sec_per_batch = float(duration)

          format_str = ('%s: step %d, losses = %.3f (%.1f examples/sec; %.3f '
                        'sec/batch)')
          print (format_str % (datetime.now(), self._step, losses_value,
                               examples_per_sec, sec_per_batch))
          f = open("log.txt",'ab')
          f.write('{0}: step {1}, losses = {2:.4f} ({3:.2f} examples/sec; {4:.2f} sec/batch)\n'.format(datetime.now(), self._step, losses_value,
                               examples_per_sec, sec_per_batch))
          f.close()
            
    with tf.train.MonitoredTrainingSession(checkpoint_dir=train_dir,
                                           hooks=[tf.train.StopAtStepHook(last_step=max_steps),
                                                  tf.train.NanTensorHook(losses),
                                                  _LoggerHook()],
                                           config=tf.ConfigProto(
                                               log_device_placement=log_device_placement)) as mon_sess:
        while not mon_sess.should_stop():
            mon_sess.run(train_op)

In [ ]:
import six

def evaluate():
  eval_batch_count = 50
  """Eval CIFAR-10 for a number of steps."""
  with tf.Graph().as_default() as g:
    # Get images and labels for CIFAR-10.
    images, labels = cifar_input.build_input('cifar10', 'cifar10/test_batch.bin', 128, 'eval')

    # Build a Graph that computes the logits predictions from the
    # inference model.
    logits = inference(images)
    saver = tf.train.Saver()
    #summary_writer = tf.summary.FileWriter(FLAGS.eval_dir)

    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    tf.train.start_queue_runners(sess)

    best_precision = 0.0
    try:
      ckpt_state = tf.train.get_checkpoint_state(train_dir)
    except tf.errors.OutOfRangeError as e:
      tf.logging.error('Cannot restore checkpoint: %s', e)
    if not (ckpt_state and ckpt_state.model_checkpoint_path):
      tf.logging.info('No model to eval yet at %s', train_dir)
    tf.logging.info('Loading checkpoint %s', ckpt_state.model_checkpoint_path)
    saver.restore(sess, ckpt_state.model_checkpoint_path)

    total_prediction, correct_prediction = 0, 0
    for _ in six.moves.range(eval_batch_count):
      (predictions, truth) = sess.run(
          [logits, labels])

      truth = np.argmax(truth, axis=1)
      predictions = np.argmax(predictions, axis=1)
      correct_prediction += np.sum(truth == predictions)
      total_prediction += predictions.shape[0]

    precision = 1.0 * correct_prediction / total_prediction
    best_precision = max(precision, best_precision)

    tf.logging.info('precision: %.3f, best precision: %.3f' %
                    (precision, best_precision))
    f = open("log.txt",'ab')
    f.write('precision: {0}, best precision: {1}'.format(precision, best_precision))
    f.close()

In [ ]:
evaluate()